### Entendimiento del Problema
“Dado que el Reino Unido (UK) fue uno de los principales países que colonizaron los
Estados Unidos (USA), y UK se encuentra en el lado este de USA, entonces hay más
ciudades/poblados con nombres de ciudades de UK en la costa este de USA en
comparación a la costa oeste”.

Despúes de hacer una pequeña búsqueda para entender el problema, llegamos a que históricamente se considera únicamente costa este de estados unidos a la **zona costera central-este y nor-este.** Esta zona fue escenario de la colonización a gran escala por parte de los británicos a partir de 1580. Para lo que respecta a este estudio serán incluidos tanto los estados de la costa este histórica, como los estados con costa en el atlántico que no sean considerados como sureños: 
* Maine (ME)
* New Hampshire (NH)
* Massachusetts (MA)
* Rhode Island (RI)
* Connecticut (CT)
* New York (NY)
* New Jersey (NJ)
* Delaware  (DE) 
* Maryland (MD)
* Virginia (VA)
* North Carolina (NC)
* South Carolina (SC)
* Georgia (GA)
* Florida (FL) 

Los estados considerados pertenecientes a la costa oeste para los efectos de este trabajo son los terrenos colindantes con la costa del océano pacífico incluyendo Arizona y Nevada (debido a motivos históricos) con la excepción de aquellos considerados sureños:

* California (CA)
* Oregon (OR)
* Washington (WA)
* Alaska (AK)
* Arizona (AZ) 
* Nevada (NV)

Si bien a grandes razgos la hipótesis es fácil de testear filtrando en la misma fuente entregada [1] usando el país Estados Unidos y el texto 'New', suponiendo claro que las ciudades empezadas con "new" en su mayoria corresponden a nombres de  ciudades de Gran Bretaña su mayor colonizador. Al hacer esto podemos observar una clara tendencia en la costa este histórica. 

Pero como queremos estar completamente seguros haremos un análisis exahustivo.

[1] https://public.opendatasoft.com/explore/dataset/worldcitiespop/table/?disjunctive.country&sort=population)

[2] https://www.50states.com/abbreviations.htm

![alt text](map_1.jpg "Filtrada usando COUNTRY ='US' y Text ='New'")

#### Hacemos una lista con los estados correspondientes a cada costa, sacados de [2]

In [1]:
east_c_l=['ME','NH','MA','RI','CT','NY','NJ','DE','MD','VA','NC','SC','GA','FL']
west_c_l=['CA','OR','WA','AK','AZ','NV']

#### Importamos Librerias

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
from time import time
num_cores = 12

#### Importamos Datasets

In [3]:
usa = dd.read_csv('usa.csv', sep=';')
uk= dd.read_csv('uk.csv',sep=';')

In [4]:
west_c=usa[usa['Region'].isin(west_c_l)]

In [5]:
east_c=usa[usa['Region'].isin(east_c_l)]

##### Exploremos un poco los datos

In [6]:
print('Nombres repetidos en la Costa Este: ',east_c.City.count().compute()-east_c.City.nunique().compute())

Nombres repetidos en la Costa Este:  10139


In [7]:
print('Nombres totales en la Costa Este: ',east_c.City.count().compute())

Nombres totales en la Costa Este:  46860


In [8]:
print('Nombres unicos en la Costa Este: ',east_c.City.nunique().compute())

Nombres unicos en la Costa Este:  36721


In [9]:
print('Nombres repetidos en la Costa Oeste: ',west_c.City.count().compute()-west_c.City.nunique().compute())

Nombres repetidos en la Costa Oeste:  1239


In [10]:
print('Nombres totales en la Costa Oeste: ',west_c.City.count().compute())

Nombres totales en la Costa Oeste:  12330


In [11]:
print('Nombres unicos en la Costa Oeste: ',west_c.City.nunique().compute())

Nombres unicos en la Costa Oeste:  11091


In [12]:
print('Nombres repetidos en UK: ',uk.City.count().compute()-uk.City.nunique().compute())

Nombres repetidos en UK:  1683


In [13]:
print('Nombres totales en UK: ',uk.City.count().compute())

Nombres totales en UK:  17708


In [14]:
print('Nombres unicos en UK: ',uk.City.nunique().compute())

Nombres unicos en UK:  16025


In [15]:
#Ciudades de Gran Bretaña que contienen la palabra york
uk[uk.City.str.contains('york',regex=True)].compute()

,Country,City,AccentCity,Region,Population,Latitude,Longitude,geopoint
443,gb,west yorkshire,West Yorkshire,G8,NaN,53.700000,-1.583333,"53.7,-1.583333"
580,gb,derryork,Derryork,S4,NaN,54.950000,-6.883333,"54.95,-6.883333"
894,gb,district of east riding of yorkshire,District of East Riding of Yorkshire,E1,NaN,53.916667,-0.500000,"53.916667,-0.5"
1386,gb,north yorkshire,North Yorkshire,J7,NaN,54.166667,-1.500000,"54.166667,-1.5"
2852,gb,west riding yorkshire,West Riding Yorkshire,B4,NaN,53.916667,-1.916667,"53.916667,-1.916667"
6260,gb,york,York,Q5,144202.0,53.954115,-1.064262,"53.954115,-1.064262"
6392,gb,newyork,Newyork,T8,NaN,56.250000,-5.283333,"56.25,-5.283333"
6545,gb,south yorkshire,South Yorkshire,L3,NaN,53.450000,-1.250000,"53.45,-1.25"
6947,gb,city of york,City of York,Q5,NaN,53.954115,-1.064262,"53.954115,-1.064262"
10863,gb,county of north yorkshire,County of North Yorkshire,J7,NaN,54.166667,-1.500000,"54.166667,-1.5"


In [16]:
# Ciudades estadounidenses que contienen la palabra york
usa[usa.City.str.contains('york',regex=True)].compute()

,Country,City,AccentCity,Region,Population,Latitude,Longitude,geopoint
1082,us,yorklyn,Yorklyn,PA,NaN,39.992222,-76.646667,"39.9922222,-76.6466667"
1443,us,york,York,AL,NaN,32.486111,-88.296389,"32.4861111,-88.2963889"
1946,us,north york,North York,WI,NaN,46.392778,-90.777500,"46.3927778,-90.7775"
2568,us,yorktown,Yorktown,IL,NaN,41.569722,-89.850000,"41.5697222,-89.85"
3788,us,york springs,York Springs,PA,NaN,40.008889,-77.115833,"40.0088889,-77.1158333"
...,...,...,...,...,...,...,...,...
135925,us,yorkwood,Yorkwood,SC,NaN,34.804167,-82.546667,"34.8041667,-82.5466667"
136366,us,york,York,ND,NaN,48.312778,-99.573056,"48.3127778,-99.5730556"
136899,us,yorktown woods,Yorktown Woods,TN,NaN,35.038333,-85.105556,"35.0383333,-85.1055556"
137081,us,yorkshire,Yorkshire,PA,NaN,39.966111,-76.661111,"39.9661111,-76.6611111"


##### Conclusiones
Podemos decir que hay muchas más ciudades en la costa este que en la costa oeste, y que en la costa este son mucho menos inmaginativos para el nombre de ciudades considerando que aprox. 21%  de los nombres están repetidos en comparación con el  aprox. 10% de la Costa Oeste.

Es necesario eliminar los nombres repetidos de la base de UK para no revisarlos dos veces cuando hagamos el match con las ciudades estadounidenses.

Ahora tenemos que solucionar un problema; anteriormente vimos'New' no se incluye como parte del nombre necesariamente, algunas tienen 'nort' o incluso sufijo de 'town' o 'ville', para colmo pasa lo mismo con algunas ciudades de Gran Bretaña. Entonces ¿cómo definir qué nombres de ciudades fueron realmente tomados de ciudades de Gran Bretaña?

Una primera aproximación a esto sería simplemente buscar entre todos los nombres ingleses que hagan match con los nombres de ciudades estadounidenses almenos parcialmente 
o una lista de stopping words

In [17]:
## Eliminamos los nombres repetidos de la base de UK
uk=uk.drop_duplicates(subset='City')
uk=uk.reset_index(drop=True)
## De Hecho solo necesitaremos el nombre de las ciudades y la población
uk=uk[['City','Population']].compute()
## Ordenamos por Población, con la intuición de que se tomaron nombres de ciudades más pobladas.
#(obviando cambios demograficos)
uk.fillna(0,inplace=True)
uk.sort_values('Population',ascending=False,inplace=True)
uk=uk[['City']]

In [18]:
#Hagamos un pandas dataframe con las ciudades
east_c_pd=east_c[['City']].compute()
west_c_pd=west_c[['City']].compute()

In [19]:
#Función de match de nombres paralelizada y distribuida.
def match(wrd,pandas):
    daskd=dd.from_pandas(pandas,npartitions=len(pandas)//100) #Por si alcanzo a implementar la disminución
    if len(daskd.City[daskd.City.str.contains(str(wrd),regex=True)].index)>0:
        return daskd.City[daskd.City.str.contains(str(wrd),regex=True)].index.compute()
    else:
        pass

In [20]:
indices_ec = Parallel(n_jobs=num_cores)(delayed(match)(wrd=city,pandas=east_c_pd) for city in uk.City)

In [21]:
indices_ec=[x for x in indices_ec if x is not None]
indices_ec = [y for x in indices_ec for y in x]
indices_ec=np.unique(indices_ec)

In [22]:
print('Tenemos ',len(indices_ec),' nombres de ciudades de la costa este tomados de UK.')

Tenemos  23024  nombres de ciudades de la costa este tomados de UK.


In [25]:
indices_wc = Parallel(n_jobs=num_cores)(delayed(match)(wrd=city,pandas=west_c_pd) for city in uk.City)

In [26]:
indices_wc=[x for x in indices_wc if x is not None]
indices_wc = [y for x in indices_wc for y in x]
indices_wc=np.unique(indices_wc)

In [27]:
print('Tenemos ',len(indices_wc),' nombres de ciudades de la costa oeste tomados de UK.')

Tenemos  4518  nombres de ciudades de la costa oeste tomados de UK.


### Conclusión
Tenemos muchos mas nombres traidos de la costa este qué de la costa oeste, sin embargo hay que considerar que hay menos ciudades en la costa este que en la costa oeste, y porsupuesto obviando los errores